# AWS SSO Configuration

This notebook guides you through setting up AWS SSO for easier admin CLI access. When done, you will be able to log into your AWS account from the cli with something like this:

```shell
export AWS_PROFILE=just-my-links
aws sso login
```

While not necessary to do it this way, it makes it easier to use AWS from your CLI. You do need CLI access to AWS if you are going to be running Terraform locally (which is very helpful for development).

Note that this is *only a good idea* for the admin account you use to inspect and interact with AWS in the course of ops duties. You do not want to (and in fact can't) use this process for CI/CD.

To do this you first need to

- Have or create an AWS account (you will need to supply a payment method though the expectation is that usage will be low enough to not get billed.)
- Log into your AWS account via the UI.
  - Ideally as part of account creation you created a superuser non-root account that you are using. Avoid using the root user whenever possible!
- Navigate to *IAM Identity Center* and click *Enable*

Now we want to set up an sso user.

The following will all be in CloudShell console in the browser. Code is denoted in markdown specifically for that reason - it is not meant to be run locally on your machine since you have not set up your cli to connect to aws yet. Instead, copy and paste it into Cloudshell within the AWS console in your browser.

When Identity Center is enabled you should be able to run this to see an instance of an organization:

```shell
aws sso-admin list-instances
```

The following assumes that you have just one instance above. If not you will want to identify what the correct one is (probably the one with "sso" in the arn) and use its arn instead of the dynamically queried one below

Lets record our instance arn, account id, and identity store id into variables within the current CLI session just so we have rapid access to them
```shell
ACCOUNT_ID=$(aws sts get-caller-identity --query "Account" --output text)
INSTANCE_ARN=$(aws sso-admin list-instances --query 'Instances[0].InstanceArn' --output text)
IDENTITY_STORE_ID=$(aws sso-admin list-instances --query 'Instances[0].IdentityStoreId' --output text)
```

Note the `IDENTITY_STORE_ID` above. Your sso instance has a url that you will visit to log in, you will need to note it. It will be something like `https://d-00000000bba.awsapps.com/start/#` you can get it with
```shell
echo "https://$IDENTITY_STORE_ID.awsapps.com/start/#"
```

You'll need this for sso configuration further down

Then create an admin permission set:
```shell
aws sso-admin create-permission-set \
  --instance-arn "$INSTANCE_ARN" \
  --name "AdministratorAccess" \
  --description "Provides full access to AWS services and resources" \
  --session-duration "PT8H"
```
Note the permission set arn noted above you will use it. Assuming it is the only one (it should be), you can set it manually the variable below or just run this

```shell
PERMISSION_SET_ARN=$(aws sso-admin list-permission-sets-provisioned-to-account \
                        --instance-arn "$INSTANCE_ARN" \
                        --account-id "$ACCOUNT_ID" \
                        --output text \
                        --query "PermissionSets[0]")
```

We then want to attach the admin access policy to this permission set.

Use this to determine if the first found permission set is indeed the admin permission set - it will say "AdministratorAccess"

```shell
aws sso-admin describe-permission-set \
  --instance-arn "$INSTANCE_ARN" \
  --permission-set-arn "$PERMISSION_SET_ARN"
```

Then if so, attach the admin policy:

```shell
aws sso-admin attach-managed-policy-to-permission-set \
  --instance-arn "$INSTANCE_ARN"
  --permission-set-arn "$PERMISSION_SET_ARN" \
  --managed-policy-arn "arn:aws:iam::aws:policy/AdministratorAccess"
```

Create an sso user for yourself. **Use your own values for email and name!** I don't want to be getting your reset links.
```shell
MY_EMAIL=gmauer@gmail.com
MY_GIVEN_NAME=George
MY_FAMILY_NAME=Mauer
aws identitystore create-user \
  --identity-store-id "$IDENTITY_STORE_ID" \
  --user-name "$MY_EMAIL" \
  --name "GivenName=$MY_GIVEN_NAME,FamilyName=$MY_FAMILY_NAME" \
  --emails "Value=$MY_EMAIL,Type=Primary" \
  --display-name "$MY_GIVEN_NAME $MY_FAMILY_NAME"
```

Now lets assign admin access to this user. Note that the below looks up the id of the user created above by username. If you have it you can just sub it in

```shell
aws sso-admin create-account-assignment \
  --instance-arn "$INSTANCE_ARN" \
  --target-id "$ACCOUNT_ID" \
  --target-type AWS_ACCOUNT \
  --permission-set-arn "$PERMISSION_SET_ARN" \
  --principal-type USER \
  --principal-id $(aws identitystore list-users --identity-store-id $"$IDENTITY_STORE_ID" --filters "AttributePath=UserName,AttributeValue=$MY_EMAIL" --query 'Users[0].UserId' --output text)
```

Finally, you will need to set up a password for this user. *This is not doable through the CLI*.
- Check your email, you will likely have to click a link to confirm it.
- Go to the IAM Identity Center
- Navigate to Users
- Select your user
- Click the "Reset Password" button

You may select the appropriate thing at this point but a straightforward recommendation is to select "generate a one-time password" and copy it to be used when logging in

For this next part you'll need the sso login url we calculated above from the `IDENTITY_STORE_ID` and your new user email/password.

*The following is in the local CLI* and you should have to do it only once:

In [ ]:
!aws configure sso

During login, reset your password [using your password manager](https://1password.com/) and ideally set up MFA. Click through menues to eventually get the AWS apps account screen. This is very useful for managing multiple aws accounts. Here you can log into a specific account as a user or view sso keys, however we're just going to set up a profile locally so you can click off of this screen.

For the following you'll need the `ACCOUNT_ID` from above and the SSO login url you just use. Create an entry in [~/.aws/config](~/.aws/config) that looks like this:

```toml
[profile just-my-links]
sso_start_url = FILL_IN_YOUR_SSO_LINK
sso_account_id = FILL_IN_YOUR_ACCOUNT_ID
sso_region = us-east-1
sso_role_name = AdministratorAccess
sso_session_name = default
output = json
region = us-east-1
```

You may then login with SSO by first setting your aws profile to use the `just-my-links` profile

In [1]:
%env AWS_PROFILE=just-my-links

env: AWS_PROFILE=just-my-links


And finally you can log in

In [ ]:
!aws sso login

A browser window will open and you will be prompted to click through a series of screens (remember to use your password manager!). You may also visit the above link manually and click through.

You will now be connected to your aws account. Note that the way `aws sso` works, your login is per-profile, not per-cli-session. You don't need to manage aws keys environment variables or anything like that. So long as the profile used is the above, you will be logged in as long as your sso session lasts. Running the following should show you your logged in user identity.

In [3]:
!aws sts get-caller-identity

{
    "UserId": "AROASDT4T2PCFCLRY32IU:gmauer@gmail.com",
    "Account": "145216492484",
    "Arn": "arn:aws:sts::145216492484:assumed-role/AWSReservedSSO_AdministratorAccess_7d6c5139d9951c91/gmauer@gmail.com"
}


This session will last as we configured in the permission set (8h above) and as long as your `aws` environment knows which profile to use (typically by ensuring the `AWS_PROFILE` environment variable is set) you will be logged in.

Be careful with this, your local cli now has admin access!

# Working with deployment

## Initial Infrastructure deployment

We deploy cloudformation in two runs. The first sets up our "initial" things that must exist and the second our lambdas that are built on top of it. We have to pass `IsFirstRun=true` like so

Note that the following will deploy a stack with the "dev" suffix implying its a dev environment

In [ ]:
!cd ./cloudformation && ./scripts/deploy.sh dev IsFirstRun=true

You can query potential errors of your stack with the following, again, note that `-dev` suffix which is due to the `dev` environment being used above

In [ ]:
!aws cloudformation describe-stack-events --stack-name just-my-links-dev --query 'StackEvents[?ResourceStatus==`CREATE_FAILED` || ResourceStatus==`ROLLBACK_IN_PROGRESS` || ResourceStatus==`ROLLBACK_COMPLETE`]' | cat

On failure, before trying again, you might need to delete your stack

In [79]:
!aws cloudformation delete-stack --stack-name just-my-links-dev

You can also check outputs of the cloudformation stack like this

In [81]:
!aws cloudformation describe-stacks --stack-name just-my-links-dev --query 'Stacks[0].Outputs'

[
    {
        "OutputKey": "ApplicationBucketName",
        "OutputValue": "just-my-links-dev-145216492484",
        "Description": "S3 Bucket name for application data (ChromaDB and logs)",
        "ExportName": "dev-just-my-links-application-bucket"
    },
    {
        "OutputKey": "EcrRepositoryUri",
        "OutputValue": "145216492484.dkr.ecr.us-east-1.amazonaws.com/dev-just-my-links-lambda",
        "Description": "ECR Repository URI"
    },
    {
        "OutputKey": "BearerTokenSecretName",
        "OutputValue": "arn:aws:secretsmanager:us-east-1:145216492484:secret:dev-just-my-links-bearer-token-QKGIdU",
        "Description": "Name of the bearer token secret in Secrets Manager",
        "ExportName": "dev-just-my-links-bearer-token-secret-name"
    }
]


At this pont we need to build and upload an image so our lambda can reference it when its deployed.

In [ ]:
!scripts/deploy-indexing-service.py

View what's been published

In [88]:
!aws ecr list-images --repository-name dev-just-my-links-lambda

{
    "imageIds": [
        {
            "imageDigest": "sha256:0c07e5d1929be7fc20ef1be1e765930dd8e6987d1972f389b99dc4ee270931d7",
            "imageTag": "latest"
        }
    ]
}


And now we can do the rest of the deploy

In [ ]:
!cd ./cloudformation && ./scripts/deploy.sh dev IsFirstRun=false

Note that because parameter values are defaulted to the previous cloudformation run, the IsFirstRun parameter *does* explicitly need to be set to `false` when this is invoked for the first time. Afterwards, the parameter can be omitted. When this deploy runs, it will give us an api gateway url or you can get one from above.

In fact, lets get them and store these values as variables in this workbook

TODO - Add in dns configuration here

In [92]:
import json
cloudformation_outputs_res = !aws cloudformation describe-stacks --stack-name just-my-links-dev --query 'Stacks[0].Outputs'
cloudformation_outputs = {x['OutputKey']: x['OutputValue'] for x in json.loads(''.join(cloudformation_outputs_res))}
application_bucket_name = cloudformation_outputs['ApplicationBucketName']
index_service_uri = cloudformation_outputs['HttpApiUrl']
index_service_arn = cloudformation_outputs['LambdaFunctionArn']
auth_token_secret_arn = cloudformation_outputs['BearerTokenSecretName']

In [93]:
import requests

response = requests.post(
    index_service_uri,
    headers={"Content-Type": "application/json"},
    json={"url": "https://example.com", "title": "Test Document"}
)
response.status_code, response.text

(401, '{"error":"Unauthorized"}')

And see the logs

In [95]:
!aws logs tail /aws/lambda/dev-just-my-links-index-document

2025-06-05T23:13:57.768000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f START RequestId: ab2287e1-0d6c-4e8f-8da2-ab2e581cf391 Version: $LATEST
2025-06-05T23:13:57.769000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f {"level":"INFO","location":"lambda_handler:110","message":"Lambda handler invoked","timestamp":"2025-06-05 23:13:57,769+0000","service":"service_undefined","cold_start":true,"function_name":"dev-just-my-links-index-document","function_memory_size":"1024","function_arn":"arn:aws:lambda:us-east-1:145216492484:function:dev-just-my-links-index-document","function_request_id":"ab2287e1-0d6c-4e8f-8da2-ab2e581cf391","correlation_id":"LtqsFi-yIAMEZ1w=","event_type":"unknown","xray_trace_id":"1-684224b3-6315e57a476fb72c3d301224"}
2025-06-05T23:13:57.770000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f {"_aws":{"Timestamp":1749165237769,"CloudWatchMetrics":[{"Namespace":"just-my-links","Dimensions":[[]],"Metrics":[{"Name":"UnauthorizedRequests"

Where did that `Unauthorized` come from? Why from our authentication of course!

It is important at this point to change our secret

In [ ]:
from uuid import uuid4
auth_token=str(uuid4())
!aws secretsmanager update-secret --secret-id "$auth_token_secret_arn" --secret-string "$auth_token"

In [98]:
response = requests.post(
    index_service_uri,
    headers={"Content-Type": "application/json", "Authorization": f"Bearer {auth_token}"},
    json={"url": "https://example.com", "title": "Test Document"}
)

response.status_code, response.text

(501,
 '{"message":"Not yet implemented - will download ChromaDB from S3, operate locally, then upload back"}')

Deploy changes to the lambda image with index-service deployment script

In [ ]:
! scripts/deploy-indexing-service.py dev

Turn on debug logging with

In [ ]:
import boto3

function_name = 'dev-just-my-links-index-document'
lambda_client = boto3.client('lambda')

response = lambda_client.get_function_configuration(FunctionName=function_name)

current_variables = {**response.get('Environment', {}).get('Variables', {}), "LOG_LEVEL": "DEBUG"}

lambda_client.update_function_configuration(FunctionName=function_name, Environment={'Variables': current_variables})

In [102]:
!aws logs tail /aws/lambda/dev-just-my-links-index-document

2025-06-05T23:13:57.768000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f START RequestId: ab2287e1-0d6c-4e8f-8da2-ab2e581cf391 Version: $LATEST
2025-06-05T23:13:57.769000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f {"level":"INFO","location":"lambda_handler:110","message":"Lambda handler invoked","timestamp":"2025-06-05 23:13:57,769+0000","service":"service_undefined","cold_start":true,"function_name":"dev-just-my-links-index-document","function_memory_size":"1024","function_arn":"arn:aws:lambda:us-east-1:145216492484:function:dev-just-my-links-index-document","function_request_id":"ab2287e1-0d6c-4e8f-8da2-ab2e581cf391","correlation_id":"LtqsFi-yIAMEZ1w=","event_type":"unknown","xray_trace_id":"1-684224b3-6315e57a476fb72c3d301224"}
2025-06-05T23:13:57.770000+00:00 2025/06/05/[$LATEST]e82281bb35a246999e6b793e72302b2f {"_aws":{"Timestamp":1749165237769,"CloudWatchMetrics":[{"Namespace":"just-my-links","Dimensions":[[]],"Metrics":[{"Name":"UnauthorizedRequests"